In [1]:
# 전처리 거친 데이터셋 불러오기
import pickle 
with open('train_data.pickle', 'rb') as fr:
    train_data = pickle.load(fr)
with open('test_data.pickle', 'rb') as fr:
    test_data = pickle.load(fr)

In [4]:
from pykospacing import spacing

In [ ]:
# 띄어쓰기 보정
train_data['doc_spaced'] = train_data['document'].swifter.apply(lambda x:spacing(x))
test_data['doc_spaced'] = test_data['document'].swifter.apply(lambda x:spacing(x))

In [32]:
# 띄어쓰기 보정 테스트
print(train_data['document'].head())
print(train_data['doc_spaced'].head())

0                                    아 더빙 진짜 짜증나네요 목소리
1                           흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 솔직히 재미는 없다평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
Name: document, dtype: object
0                                    아 더빙 진짜 짜증나네요 목소리
1                       흠포스터 보고 초딩 영화 줄오버 연기조차 가볍지 않구나
2                               너무 재밓었다 그래서 보는 것을 추천한다
3                           교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4    사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...
Name: doc_spaced, dtype: object


In [8]:
def tokenize(sentence):
    """
    형태소로 분리 및 조사는 제거
    """
    tokenizer = Okt()
    morphs_pos = tokenizer.pos(sentence)
    return [mp[0] for mp in morphs_pos if mp[1] != 'Josa']

In [26]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [14]:
# 데이터셋 토큰화하기 (Control)
train_data['doc_cleaned'] = train_data['document'].progress_apply(tokenize)
test_data['doc_cleaned'] = test_data['document'].progress_apply(tokenize)

# 데이터셋 토큰화하기 (Spaced)
train_data['doc_cleaned_spaced'] = train_data['doc_spaced'].progress_apply(tokenize)
test_data['doc_cleaned_spaced'] = test_data['doc_spaced'].progress_apply(tokenize)

In [29]:
def spacingABtest(doc_name):
    train = train_data[doc_name].values
    test = test_data[doc_name].values

    y_train = train_data['label'].values
    y_test = test_data['label'].values

    # TFIDF 주요 단어 추출 
    X_train = np.array(train)
    train_pos = X_train[y_train == 1]
    train_neg = X_train[y_train == 0]

    tfidfv = TfidfVectorizer(preprocessor=' '.join, max_features=30000).fit(train_pos)
    score = {key : value for key, value in zip(sorted(tfidfv.vocabulary_, key = lambda x : tfidfv.vocabulary_[x]), tfidfv.transform(train_pos).toarray().sum(axis=0))}
    top_pos = sorted(score, key = lambda x : score[x],reverse=True)[:10000]

    tfidfv = TfidfVectorizer(preprocessor=' '.join, max_features=30000).fit(train_neg)
    score = {key : value for key, value in zip(sorted(tfidfv.vocabulary_, key = lambda x : tfidfv.vocabulary_[x]), tfidfv.transform(train_neg).toarray().sum(axis=0))}
    top_neg = sorted(score, key = lambda x : score[x],reverse=True)[:10000]

    top = top_pos + top_neg

    # Countervectorize 
    train = list(map(' '.join, train))
    test = list(map(' '.join, test))

    vectorizer = CountVectorizer()
    vectorizer.fit(top)
    X_train = vectorizer.transform(train).toarray()
    X_test = vectorizer.transform(test).toarray()

    # 성능 테스트 
    nbc = MultinomialNB()
    nbc.fit(X_train, y_train)
    y_pred = nbc.predict(X_test)
    
    return accuracy_score(y_test, y_pred)

In [30]:
name_control = 'doc_cleaned'
name_spaced = 'doc_cleaned_spaced'

print("테스트 정확도(Control): {:.3f}".format(spacingABtest(name_control)))
print("테스트 정확도(Spaced): {:.3f}".format(spacingABtest(name_spaced)))

테스트 정확도(Control): 0.832
테스트 정확도(Spaced): 0.830
